<a href="https://colab.research.google.com/github/michellejm/sentiment_book_reviews/blob/main/hf_tutorial_imdb_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with finetuned distilBERT
Following the [HuggingFace tutorial](https://huggingface.co/docs/transformers/en/tasks/sequence_classification) on text classification using [Goodreads data](https://www.kaggle.com/competitions/goodreads-books-reviews-290312)

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
pip install transformers datasets evaluate accelerate

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metad

In [17]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import pipeline

from datasets import load_dataset

import evaluate

from sklearn.model_selection import train_test_split


## Goodreads dataset

In [ ]:
df = pd.read_csv("data/goodreads_train.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/goodreads_train.csv'

In [ ]:
df.describe()

In [ ]:
# transform the rating column to a label column

def rating_to_sent(rating):
    # rating 3 is assigned positive because sampling a few reviews,
    # I personally would call most of them more positive than negative. This could be changed
    if rating > 3:
        return 1
    elif rating == 3:
        return 1
    elif rating < 3:
        return 0

df['label'] = df['rating'].apply(rating_to_sent)

In [ ]:
# rename column
df.rename(columns={'review_text':'text'}, inplace=True)

#trim dataframe to just text and label
hf_df = df[['text', 'label']]

#save as csv for later and read that csv into the goodreads dataset we'll use
hf_df.to_csv("data/goodreads_for_hf.csv")
#gr_df = pd.read_csv("data/goodreads_for_hf.csv")

gr_ds = load_dataset("csv", data_files="data/goodreads_for_hf.csv")
type(gr_ds)

## Try w IMDB

In [4]:
from datasets import load_dataset

imdb = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Preprocessing

In [5]:
#tokenize w BERT's tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
#truncate input to the maximum length of BERT's input length

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation = True)

#tokenized_gr = gr_ds.map(preprocess_function, batched = True)
#type(tokenized_gr)

In [7]:
tokenized_imdb = imdb.map(preprocess_function, batched = True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
# now pad the inputs so that each batch is the same length
# do this by batches because otherwise takes too long and uses unnecessary resources
# tbh this function itself is a total black box

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
type(data_collator)

transformers.data.data_collator.DataCollatorWithPadding

In [10]:
# measure accuracy while training

accuracy = evaluate.load("accuracy")

In [11]:
#pass predictions and labels to `compute` to calculate the accuracy

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return accuracy.compute(predictions = predictions, references = labels)

In [12]:
#map ids to labels
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}


In [13]:
# establish the model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# define training arguments

training_args = TrainingArguments(
    output_dir = "models",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb['train'],
    eval_dataset=tokenized_imdb['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()



wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: michelleamcsweeney (michelleamcsweeney-the-city-university-of-new-york) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.223900,0.230147,0.911280
2,0.142800,0.231459,0.931080


TrainOutput(global_step=3126, training_loss=0.20655757955305865, metrics={'train_runtime': 2045.6985, 'train_samples_per_second': 24.442, 'train_steps_per_second': 1.528, 'total_flos': 6556904415524352.0, 'train_loss': 0.20655757955305865, 'epoch': 2.0})

In [15]:
#add to my HuggingFace (there must be a million of these)
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/linguanne/models/commit/6ed8e1110dc008f1c590044ad0ea5f11a7c32886', commit_message='End of training', commit_description='', oid='6ed8e1110dc008f1c590044ad0ea5f11a7c32886', pr_url=None, repo_url=RepoUrl('https://huggingface.co/linguanne/models', endpoint='https://huggingface.co', repo_type='model', repo_id='linguanne/models'), pr_revision=None, pr_num=None)

In [18]:
# my honest review of My Brilliant Friend

text1 = "This is good -- I love listening to the Italian, and the acting is really good. It's hard for a film to be as good as the books, but I think My Brilliant Friend has done it."

In [20]:
#use my model
classifier = pipeline("sentiment-analysis", model = "linguanne/models")
classifier(text1)

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9920943379402161}]

In [21]:
text2 = "too terrifying! I cannot take the psychological horror that is any of the Squid Games."
classifier = pipeline("sentiment-analysis", model = "linguanne/models")
classifier(text2)

Device set to use cuda:0


[{'label': 'NEGATIVE', 'score': 0.9715889692306519}]